# Tools Langchain Agent

In [21]:
# Install necessary libraries
%pip install langchain langchainhub arxiv wikipedia langchain_community faiss-cpu langchain_core langchain_community langchain_openai langserve

Note: you may need to restart the kernel to use updated packages.


In [22]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, tool
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools.retriever import create_retriever_tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import  ArxivQueryRun
from langchain import hub

llm = ChatOllama(
    model="llama3.1",
    # format="json",
    keep_alive=-1, # keep the model loaded indefinitely
    temperature=0.1,
    max_new_tokens=512
    )

In [23]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

wiki.name

'wikipedia'

In [24]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb = FAISS.from_documents(
    documents=docs[:30],
     embedding=OllamaEmbeddings(model="dolphin-llama3",show_progress=True),
)
retriever = vectordb.as_retriever()
retriever

OllamaEmbeddings: 100%|██████████| 1/1 [00:02<00:00,  2.92s/it]


VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x117e2e7d0>)

In [25]:
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

retriever_tool.name

'langsmith_search'

In [26]:
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [27]:
tools = [wiki, arxiv, retriever_tool]

tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/gsampaio/redhat/ai/llm-agents/.conda/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x10f6922a0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.

In [28]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
# prompt = hub.pull("klcoder/mistral-functioncalling")
prompt.messages
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [29]:
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input":"Tell me about Langsmith"})

# # Using with chat history
# from langchain_core.messages import AIMessage, HumanMessage
# agent_executor.invoke(
#     {
#         "input": "what's my name?",
#         "chat_history": [
#             HumanMessage(content="hi! my name is bob"),
#             AIMessage(content="Hello Bob! How can I assist you today?"),
#         ],
#     }
# )



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'information about LangSmith'}`




OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00,  4.09it/s]







Get started with LangSmith | 🦜️🛠️ LangSmith







Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We provide multiple ways to log traces to LangSmith. Below, we'll highlight
how to use 

{'input': 'Tell me about Langsmith',
 'output': 'Based on the tool call response, here is a formatted answer to the original question:\n\n**What is Langsmith?**\n\nLangsmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\n\n**Key Features:**\n\n1. **Tracing**: Langsmith provides multiple ways to log traces to the platform.\n2. **Evaluation**: Evaluation requires a system to test, data to serve as test cases, and optionally evaluators to grade the results.\n3. **Accuracy Evaluator**: A built-in accuracy evaluator is available for use.\n\n**Getting Started:**\n\n1. Install Langsmith using pip or yarn.\n2. Create an API key on the Settings page.\n3. Set up your environment by exporting necessary variables.\n4. Log your first trace using one of the provided methods.\n5. Run your first evaluation using a built-in accuracy evaluator.\n\nI hope thi